### Imports

In [1]:

import xgboost as xg
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from matplotlib import pyplot

/Users/codyreece/Desktop/Repos/airbnb/air-scrape/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# !pip install --upgrade category_encoders rich catboost
from rich.console import Console
console = Console()
print = console.print
from wrangling import X, y

In [3]:
import time
import math
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import matplotlib.pyplot as plt

import yellowbrick as yb
from yellowbrick.features import Rank1D
from yellowbrick.regressor import AlphaSelection, PredictionError, ResidualsPlot
from yellowbrick.datasets import load_energy
from yellowbrick.model_selection import ValidationCurve
from yellowbrick.style import set_palette

import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lars
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import OrthogonalMatchingPursuit

from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression
from sklearn import linear_model
from sklearn.linear_model import Lasso

import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
import catboost as ctb

from hyperopt import STATUS_OK, STATUS_FAIL, Trials, fmin, hp, tpe

np.set_printoptions(precision=3, suppress=True)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=1)

In [5]:
plt.style.context('dark_background')
set_palette('sns_bright')

cm = sns.color_palette("blend:white,#00ff77", as_cmap=True)

def headd(i):
    return i.style.background_gradient(cmap = cm,axis=None)

### Visuals

In [ ]:
## Ranking the features

fig, ax = plt.subplots(1, figsize=(10, 35))
vzr = Rank1D(ax=ax, color='#00ff77')
vzr.fit(X_train, y_train)
vzr.transform(X_train)
sns.despine(left=True, bottom=True)
vzr.poof();

In [ ]:
X_outliers = pd.DataFrame(index=X.columns, columns=['outliers', 'outliers%'])

for col in X.columns:
    if any(x in str(X[col].dtype)for x in ['int', 'float', 'int64', 'uint8']):
        
        X_outliers.loc[col, 'count'] = len(X)
        X_outliers.loc[col, 'q1'] = X[col].quantile(0.25)
        X_outliers.loc[col, 'q3'] = X[col].quantile(0.75)
        X_outliers.loc[col, 'iqr'] = X_outliers.loc[col, 'q3'] - X_outliers.loc[col, 'q1']
        X_outliers.loc[col, 'lower'] = X_outliers.loc[col, 'q1'] - (3 * X_outliers.loc[col, 'iqr'])
        X_outliers.loc[col, 'upper'] = X_outliers.loc[col, 'q3'] + (3 * X_outliers.loc[col, 'iqr'])
        X_outliers.loc[col, 'min'] = X[col].min()
        X_outliers.loc[col, 'max'] = X[col].max()
        X_outliers.loc[col, 'outliers'] = ((X[col] < X_outliers.loc[col, 'lower']) | (X[col] > X_outliers.loc[col,'upper'])).sum()
        X_outliers.loc[col, 'outliers%'] = np.round(X_outliers.loc[col,
        'outliers'] / len(X) *100)
        
headd(X_outliers.head(10))

In [ ]:
#Distribution of price
%matplotlib inline

fig, axs = plt.subplots(ncols=2, figsize=(14, 4))
fig.suptitle('Distribution of max guests (before and after removing large listings > 10)', weight='bold', fontsize=12)

# Before cleaning
x_axis=X['numberOfGuests'].dropna()
sns.distplot(pd.Series(x_axis, name='Max guests (before cleaning)'), ax=axs[0])

# Remove where price > 1000
condition = X[X['numberOfGuests'] > 400]
rows_to_drop = condition.index
print("You dropped {} rows.".format(condition.shape[0]))
X = X.drop(rows_to_drop, axis=0)
print("Dataset has {} rows, {} columns.".format(*X.shape))

#After cleaning
x_axis=X['numberOfGuests'].dropna()
sns.distplot(pd.Series(x_axis, name='Max guests (after cleaning)'), ax=axs[1]);

In [ ]:
## Adding est. Annual Revenue
print("Dataset has {} rows, {}  before engineering.".format(*X.shape))
avg_occupancy_per_week = 4
X['yield'] = avg_occupancy_per_week * y  * 52

# cols_to_drop = ['cleaning_fee']
# df = df.drop(cols_to_drop, axis = 1)
print("Dataset has {} rows, {} columns.".format(*X.shape))


### Linear Regression

In [ ]:
model1 = LinearRegression()
model1.fit(X_train, y_train)

In [ ]:
y_pred = model1.predict(X_test)

lr=r2_score(y_test, y_pred)
lr

In [ ]:
training_score=model1.score(X_train,y_train)
training_score

In [ ]:
# model evaluation for testing set

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


print("The model performance for testing set")
print("--------------------------------------")
print('MAE:  {}'.format(round(mae)))
print('MSE:  {}'.format(round(mse)))
print('RMSE: {}'.format(round(rmse)))
print('R2:   {}'.format(round(r2, 3)))

In [ ]:
lr_price= pd.DataFrame({'Actual':y_test,'Predict':y_pred})
lr_price.head()

In [ ]:
# Density plot for lr

lr_reg = lr_price.plot.kde()

### XGBRegressor

In [ ]:
model2=XGBRegressor(silent=True)
model2.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import r2_score,classification_report

y_pred = model2.predict(X_test)

xgb=(r2_score(y_test, y_pred))
xgb

In [ ]:
training_score=model2.score(X_train,y_train)
training_score

In [ ]:
# model evaluation for testing set

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


print("The model performance for testing set")
print("--------------------------------------")
print('MAE:  {}'.format(round(mae)))
print('MSE:  {}'.format(round(mse)))
print('RMSE: {}'.format(round(rmse)))
print('R2:   {}'.format(round(r2, 3)))

In [ ]:
xgb_price= pd.DataFrame({'Actual':y_test,'Predict':y_pred})
xgb_price.head()

In [ ]:
xgb_reg = xgb_price.plot.kde()

### Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
model3=Ridge()
model3.fit(X_train,y_train)

In [ ]:
y_pred = model3.predict(X_test)

rr=r2_score(y_test, y_pred)
rr

In [ ]:
training_score=model3.score(X_train,y_train)
training_score

In [ ]:
# model evaluation for testing set

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


print("The model performance for testing set")
print("--------------------------------------")
print('MAE:  {}'.format(round(mae)))
print('MSE:  {}'.format(round(mse)))
print('RMSE: {}'.format(round(rmse)))
print('R2:   {}'.format(round(r2, 3)))

In [ ]:
rr_price= pd.DataFrame({'Actual':y_test,'Predict':y_pred})
rr_price.head()

In [ ]:
rr_reg = rr_price.plot.kde()

### Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model4=RandomForestRegressor()
model4.fit(X_train,y_train)

In [ ]:
y_pred = model4.predict(X_test)

rf=r2_score(y_test, y_pred)
rf

In [ ]:
training_score=model4.score(X_train,y_train)
training_score

In [ ]:
# model evaluation for testing set

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)


print("The model performance for testing set")
print("--------------------------------------")
print('MAE:  {}'.format(round(mae)))
print('MSE:  {}'.format(round(mse)))
print('RMSE: {}'.format(round(rmse)))
print('R2:   {}'.format(round(r2, 3)))

In [ ]:
rf_price= pd.DataFrame({'Actual':y_test,'Predict':y_pred})
rf_price.head()

In [ ]:
rf_reg = rf_price.plot.kde()

### ExtraTreeRegressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model5=ExtraTreesRegressor()
model5.fit(X_train,y_train)

In [ ]:
y_pred = model5.predict(X_test)
xtr=(r2_score(y_test, y_pred))
xtr

In [ ]:
xtr_train_score=model5.score(X_train,y_train)
xtr_train_score

In [ ]:
xtr_price= pd.DataFrame({'Actual':y_test,'Predict':y_pred})
xtr_price.head()

In [ ]:
xt_reg = xtr_price.plot.kde()

### Analyzing all Model Performances

In [ ]:
score= [
        ['Linear Regression', lr],
        ['Ridge Regression', rr],
        ['XGBoost Regressor', xgb],
        ['Xtra tree Regressor', xtr],
        ['RandomForestRegressor', rf]
        ]
 
predicted_pernight = pd.DataFrame(score, columns = ['Regression models', 'accuracy_scores'])
predicted_pernight

In [ ]:
plt.figure(figsize=(7,5))
plt.xticks(rotation=90)
plt.title('Regression models & accuracy scores',fontsize=20)
sns.barplot(x="Regression models", y="accuracy_scores",  data=predicted_pernight ,palette = "blend:#d900ff,#00ff77")

In [ ]:
# Scatter plot for the results

plt.figure(figsize = (20,4))
ax = plt.subplot(1,5,1)
ax = plt.subplot(1,5,1).set_title('LinearRegression')
sns.scatterplot(x=lr_price.Actual, y=lr_price.Predict)
ax = plt.subplot(1,5,2)
ax = plt.subplot(1,5,2).set_title('RandomForestRegressor')
sns.scatterplot(x=rf_price.Actual, y=rf_price.Predict)
ax = plt.subplot(1,5,3)
ax = plt.subplot(1,5,3).set_title('XGBRegressor')
sns.scatterplot(x=xgb_price.Actual, y=xgb_price.Predict)
ax = plt.subplot(1,5,4)
ax = plt.subplot(1,5,4).set_title('RidgeRegression')
sns.scatterplot(x=rr_price.Actual, y=rr_price.Predict)
ax = plt.subplot(1,5,5)
ax = plt.subplot(1,5,5).set_title('Xtra tree Regressor')
sns.scatterplot(x=xtr_price.Actual, y=xtr_price.Predict);

In [ ]:
for i in [model1, model2, model3, model4, model5]:
  visualizer = ResidualsPlot(
      i,
      hist=False,
      qqplot=True,
      size=(500, 300),
      train_color="indigo",
      test_color="#00ff77", 
      )

  visualizer.fit(X_train, y_train)
  visualizer.score(X_test, y_test)
  g = visualizer.poof();

### XGB Regression

In [ ]:
X_train.shape

In [ ]:
import xgboost as xgb
housing_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

# Creata the parameter dictionary for each tree: params
params = {"objective":"reg:squarederror", "max_depth":4}

# Create list of number of boosting rounds
num_rounds = [5, 10, 15, 20, 30, 40, 50, 60, 100, 500]

# Empty list to store final round rmse per XGBoost model
final_rmse_per_round = []

# Interate over num_rounds and build one model per num_boost_round parameter
for curr_num_rounds in num_rounds:
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=10, 
                        num_boost_round=curr_num_rounds, metrics='rmse', 
                        as_pandas=True, seed=123)
    
    # Append final round RMSE
    final_rmse_per_round.append(cv_results['test-rmse-mean'].tail().values[-1])
    
# Print the result DataFrame
num_rounds_rmses = list(zip(num_rounds, final_rmse_per_round))
print(pd.DataFrame(num_rounds_rmses, columns=['num_boosting_rounds', 'rmse']))

In [ ]:
# Perform cross-validation with early-stopping: cv_results
cv_results = xgb.cv(
    dtrain=housing_dmatrix, 
    nfold=15,
    params=params,
    metrics="rmse",
    early_stopping_rounds=40,
    num_boost_round=500,
    as_pandas=True,
    seed=1
    )

# Print cv_results
print(cv_results)

In [ ]:
eta_vals = [0.001, 0.005, .01, 0.1, .2, .3, .4, .5, .55, .57, .59, .6, .75, 1]
best_rmse = []

# Systematicallyvary the eta
for curr_val in eta_vals:
    params['eta'] = curr_val
    
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=30,
                        early_stopping_rounds=40, num_boost_round=10, metrics='rmse', seed=123, 
                       as_pandas=True)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results['test-rmse-mean'].tail().values[-1])
    
# Print the result DataFrame
print(pd.DataFrame(list(zip(eta_vals, best_rmse)), columns=['eta', 'best_rmse']))

In [ ]:
# Create list of hyperparameter values: colsample_bytree_vals
colsample_bytree_vals = [0.001, 0.005, .01, 0.1, .2, .21, .22, .23, .24, .3, .4, .5, .55, .57, .59, .6, .75, 1]
best_rmse = []

# Systematically vary the hyperparameter value 
for curr_val in colsample_bytree_vals:
    params['colsample_bytree'] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=2,
                 num_boost_round=10, early_stopping_rounds=5,
                 metrics="rmse", as_pandas=True, seed=123)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(colsample_bytree_vals, best_rmse)), 
                   columns=["colsample_bytree","best_rmse"]))

In [ ]:
# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'colsample_bytree': [.5, 0.7, 1],
    'n_estimators': [100, 200, 350, 500],
    'max_depth': [2, 5, 7, 10]
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor()

# Perform grid search: grid_mse
grid_mse = GridSearchCV(param_grid=gbm_param_grid, estimator=gbm, 
                        scoring='neg_mean_squared_error', cv=4, verbose=3)

# Fit grid_mse to the data
grid_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'n_estimators': [25, 50, 75, 100],
    'max_depth': range(2, 16, 2)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(n_estimators=10)

# Perform random search: randomized_mse
randomized_mse = RandomizedSearchCV(param_distributions=gbm_param_grid, estimator=gbm, 
                                    scoring='neg_mean_squared_error', n_iter=5, cv=4, 
                                   verbose=2)

# Fit randomized_mse to the data
randomized_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

### Other models